In [2]:
import LibACCPS as lib 
fz=frozenset

e_obs=fz(["a","c", "d", "g"])
e_unobs=fz(["b" ])
e_ins={"c", "d"}
e_era={"c", "g"}
g=lib.DFA(
    states=fz(["0","1","2", "3","4","5","6","7","8"]),
    alphabet=fz(["a","b","c", "d", "g"]),
    initial="0",

    delta={
        ("0","a") : "1",
        ("1","b") : "2",
        ("2","a") : "5",
        ("2","c") : "3",
        ("3","a") : "4",
        ("4","g") : "4",
        ("5","c") : "6",
        ("6","a") : "7",
        ("7","b") : "8",
        ("8","d") : "8",

        # ("2","g") : "1",
        #("3","d") : "1",
        ("4","d") : "3",
        # ("5","g") : "1",
        ("3","d") : "4",
        # ("4","c") : "2",
        # ("8","g") : "4",
    }    
)



eq_nfa = lib.mask(g, e_obs, e_unobs)

#eq_nfa.print_data()
lib.draw_dfa_graphviz(g, filename="DFA")




In [3]:
obs=lib.compute_observer(eq_nfa)
#obs.print_data()


#lib.draw_dfa_graphviz(obs, filename="obs_graph")

In [4]:
att_obs, e_plus, e_minus=lib.create_attack_observer(obs, e_ins, e_era)

#lib.draw_dfa_graphviz(att_obs, filename="att_obs_graph")
print("Attack Observer")
att_obs.print_data()

Attack Observer
----- STATES -----
{frozenset({'5'}), frozenset({'3'}), frozenset({'1', '2'}), frozenset({'8'}), frozenset({'4'}), frozenset({'8', '7'}), frozenset({'0'}), frozenset({'6'})}
---- ALPHABET ----
frozenset({'d', 'c+', 'c-', 'g-', 'd+', 'a', 'c', 'g'})
--- INIT STATE ---
frozenset({'0'})
----- DELTA ------
frozenset({'0'}) --\a\--> frozenset({'1', '2'})
frozenset({'1', '2'}) --\a\--> frozenset({'5'})
frozenset({'1', '2'}) --\c\--> frozenset({'3'})
frozenset({'5'}) --\c\--> frozenset({'6'})
frozenset({'3'}) --\d\--> frozenset({'4'})
frozenset({'3'}) --\a\--> frozenset({'4'})
frozenset({'6'}) --\a\--> frozenset({'8', '7'})
frozenset({'4'}) --\d\--> frozenset({'3'})
frozenset({'4'}) --\g\--> frozenset({'4'})
frozenset({'8', '7'}) --\d\--> frozenset({'8'})
frozenset({'8'}) --\d\--> frozenset({'8'})
frozenset({'5'}) --\d+\--> frozenset({'5'})
frozenset({'3'}) --\d+\--> frozenset({'3'})
frozenset({'1', '2'}) --\d+\--> frozenset({'1', '2'})
frozenset({'8'}) --\d+\--> frozenset({'8

In [5]:
op_obs=lib.create_operator_observer(obs, e_ins, e_era)

#print("Op Observer")
#op_obs.print_data()
lib.draw_dfa_graphviz(op_obs, filename="op_obs_graph")

In [6]:
mega_obs=lib.concurrent_composition(att_obs, op_obs)

#mega_obs.print_data()

lib.draw_dfa_graphviz(mega_obs, filename="joint_observer_graph")


In [7]:

#g1 = lib.gn_creator(1, att_obs.alphabet, e_plus)
#g1.print_data()

#lib.draw_dfa_graphviz(g1, filename="1_bounded_attack")


#bounded_1=lib.concurrent_composition(mega_obs, g1)
#lib.draw_dfa_graphviz(bounded_1, filename="1_bounded_joint")


In [1]:



forbidden_states=set(lib.compute_forbidden(mega_obs))
trim, R_preemp=lib.trim_joint_observer_v2(mega_obs, e_obs,e_era, e_ins)
weakly=mega_obs.states-trim.states-forbidden_states
colors={
    "red": forbidden_states,
    "orange": weakly,
    "green": R_preemp
}

lib.draw_dfa_graphviz(mega_obs, filename="Trim_Colored", state_colors=colors)
lib.draw_dfa_graphviz(trim, filename="Trim", state_colors=colors)



NameError: name 'lib' is not defined

In [12]:
import random

e_c=[]
costi={}
random.seed(2219)
for e in e_ins:
    e_plus=e+"+"
    e_c.append(e_plus)
    costi[e_plus] = random.randint(5, 7)



for e in e_era:
    em=e + "-"
    e_c.append(em)
    costi[em]= random.randint(1, 3)

print("Costi:", costi)
MisRelation=set([tuple([frozenset(["8"]), frozenset(["4"])])])
e_c=set(e_c)
e_uc=trim.alphabet-e_c
Super=lib.SupervisorV2(
    g=trim,
    e_c = e_c,
    e_uc = e_uc,
    MisRelation=MisRelation,
    costi=costi
)

costo, eliminati = Super.OptimalSupervisor()

print("Costo:", costo)
print("Eliminati:", eliminati)


Costi: {'d+': 7, 'c+': 5, 'c-': 1, 'g-': 1}
Costo: 2
Eliminati: {("(frozenset({'5'}), frozenset({'3'}))", 'c-'), ("(frozenset({'5'}), frozenset({'4'}))", 'c-')}


In [10]:
def state_elimination(G, targets) -> str:
    # Crea un dizionario per memorizzare le transizioni come espressioni regolari
    # La chiave è una tupla (stato_partenza, stato_arrivo)
    # Il valore è l'espressione regolare che rappresenta tutte le transizioni tra questi stati
    transitions= {}
    
    # Converte tutte le transizioni dell'automa in espressioni regolari base
    for (s, e), d in G.delta.items():
        key = (s, d)  # Chiave: (stato_partenza, stato_arrivo)
        label = str(e)  # Converti l'evento in stringa
        if key in transitions:
            # Se esiste già una transizione tra questi stati, aggiungi l'alternativa
            transitions[key] = f"({transitions[key]})|({label})"
        else:
            # Altrimenti, crea una nuova transizione
            transitions[key] = label

    # Copia gli stati dell'automa
    states = set(G.states)
    initial = G.initial
    finals = set(targets)

    # Crea stati fittizi iniziale e finale
    # Aggiunge '_' finché non trova nomi non usati
    q_init = "q_init"
    q_final = "q_final"
    while q_init in states or q_final in states:
        q_init += "_"
        q_final += "_"

    # Aggiunge i nuovi stati e le transizioni epsilon
    states.update({q_init, q_final})
    transitions[(q_init, initial)] = ""  # Transizione ε dallo stato fittizio all'iniziale
    for f in finals:
        transitions[(f, q_final)] = ""  # Transizioni ε dai finali allo stato fittizio finale

    # Aggiorna iniziale e finali con i nuovi stati fittizi
    initial = q_init
    finals = {q_final}

    # Identifica gli stati da eliminare (tutti tranne iniziale e finale fittizi)
    to_eliminate = states - {initial} - finals

    # Algoritmo di eliminazione degli stati
    while to_eliminate:
        rip = to_eliminate.pop()  # Prende uno stato da eliminare

        # Trova tutte le transizioni entranti nello stato da eliminare
        in_trans = {}
        for (i, a) in transitions:
            if a == rip and i != rip:
                in_trans[(i, a)] = transitions[(i, rip)]

        # Trova tutte le transizioni uscenti dallo stato da eliminare
        out_trans = {}
        for (b, j) in transitions:
            if b == rip and j != rip:
                out_trans[(b, j)] = transitions[(rip, j)]

        loop = transitions.get((rip, rip), '')

        for (i, _), in_regex in in_trans.items():
            for (_, j), out_regex in out_trans.items():
                mid = f"({loop})*" if loop else ''
                new_regex = f"{in_regex}{mid}{out_regex}"
                old = transitions.get((i, j), '')
                if old:
                    transitions[(i, j)] = f"({old})|({new_regex})"
                else:
                    transitions[(i, j)] = new_regex

        # Rimuovi le transizioni coinvolgenti lo stato eliminato
        new_transitions = {}
        for (i, j), r in transitions.items():
            if i != rip and j != rip:
                new_transitions[(i, j)] = r
        transitions = new_transitions

    # Costruisci la regex finale (iniziale -> finali)
    regexes = []
    for f in finals:
        if (initial, f) in transitions:
            regexes.append(transitions.get((initial, f), ''))
    final_regex = '|'.join(f"({r})" for r in regexes if r)
    return final_regex or ""

print(state_elimination(Super.trimmed, set([tuple([frozenset(["6"]), frozenset(["3"])])])))

(ac+(d+d+)*a(d+d+)*(d+c-)|(c-d+))


In [11]:
e_plus=[]
rel={}
for e in e_ins:
    e_plus=e+"+"
    rel[e_plus] = e

e_minus=[]
for e in e_era:
    em=e + "-"
    e_minus.append(em)

obs_NFA = lib.mask(trim, e_obs,  e_minus, {}, rel)

controlled=lib.compute_observer(obs_NFA)
lib.draw_dfa_graphviz(controlled, filename="ObsV2", state_colors=colors)